# 导入模块

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,datetime
import re

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [9]:

pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [10]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [11]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
,om.buy_service_product,tso.status as service_status 
,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
, tor.update_time, tomt.reason, tprm.purchase_amount, tprm.liquidated_damages 违约金
, om.order_type, om.union_rent_tag, tojo.app_type, tprm.rent_received 已收租金 
, tolog.status 物流状态
,replace(case when JSON_VALID(risk.data) THEN JSON_EXTRACT(risk.data, '$.rejected_ato_risk') end,'"','') as rejected_ato_risk
,tp.classify_id, tpmn.name 机型
from  db_digua_business.t_order  om
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 

where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1 
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
-- 京东外部订单关联表
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
-- 订单物流表
left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
-- 租后分期表
-- left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
left join db_credit.risk risk on risk.order_id=om.id
-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB

left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id


where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and pa.type!=4
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )               -- 近20天数据
-- and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-06-09'
-- and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-05'
-- and hour(om.create_time) >= 11
-- and hour(om.create_time) < 17
-- and TIME(om.create_time) <= '17:30:00'
'''

df_order = clean.query(sql1)
df_order.shape

(298691, 54)

In [12]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)

# 数据处理

In [13]:
def process_data(df_order):
    # 复制数据
    df = df_order.copy()
    # df = df[df.type!=4]
    
    # 处理日期
    df["下单日期"]=df["create_time"].dt.date
    df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
    df["月份"]=df["下单日期"].dt.month
    df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
    df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
    df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
    df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
    df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
    df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
    df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

    # 处理备注信息
    # df['取消原因2'] = df['cancel_reason'].str.split('：').apply(lambda x: x[-1] if x is not None else x)
    def getcolor(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="颜色":
                return color_list[j]["value"]
    df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)


    def getneicun(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="内存":
                return color_list[j]["value"]

    df.loc[:,"内存"]=df.apply(lambda x: getneicun(x["sku_attributes"]),axis=1)
    def getfangan(s):
        color_list=json.loads(s)
        for j in range(0,len(color_list)):
            if color_list[j]["key"]=="租赁方案":
                return color_list[j]["value"]
    df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)

    df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")


    df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
    df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
    df.loc[:,'机型内存'] = df['机型'] +'_'+ df['内存']
    
    df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
    
    df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)
    
    # 订单去重
    df = clean.order_drop_duplicates(df)
    
    # 定义状态
    df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
    df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
    df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)
    
    df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
    
    # 保留、剔除商家数据
    df_contain = df.copy()
    df = clean.drop_merchant(df)
    # 去重订单明细导出
    # today = datetime.today() 
    # t_date = today.date().strftime('%Y-%m-%d')
    
    df_j = df[df["进件"]=="进件"]
    
    df = clean.status_node(df)
    
    # 剔除拒量订单
    df2 = df.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量==0]
    day_name_cn_mapping = {
        'Monday': '星期一',
        'Tuesday': '星期二',
        'Wednesday': '星期三',
        'Thursday': '星期四',
        'Friday': '星期五',
        'Saturday': '星期六',
        'Sunday': '星期日'
    }
    
    return df, df2

# 合并数据

In [14]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(298687)

# 处理日期

In [15]:
# df[df['order_id'] == 3909094]

In [16]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df['date']=df['create_time'].dt.date
df['hour']=df['create_time'].dt.hour
df.shape

(298687, 58)

In [17]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                       100483
命中蚁盾分3.0强拒                                              36479
命中冰鉴青云分强拒                                               30322
命中TD212强拒                                               21931
命中强拒加入临时黑名单                                             16768
                                                        ...  
命中融安分低于680强拒, 评分等级低于D,命中拒绝, 命中全景雷达组合规则1強拒, 命中自有模型强拒         1
评分等级低于D,命中拒绝, 命中score_201048強拒, 命中B003_fico强拒               1
命中TD212强拒, 命中模型2501强拒, 命中策略240703_14强拒                      1
综合风险等级为3, 命中细化综合风险等级强拒, 命中模型2508强拒                          1
评分等级低于D,命中拒绝, 命中模型2508组合强拒                                  1
Name: count, Length: 1043, dtype: int64

# 处理备注信息

In [18]:
# df["total_describes"].str.split("用户无法联系：").str[1].astype(str).str.split("$").str[0].str.strip()

In [19]:
df = df[df['sku_attributes'].notnull()]
df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
# df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
# df["内存"].value_counts() 

def getfangan(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="租赁方案":
            return color_list[j]["value"]
df.loc[:,"租赁方案"]=df.apply(lambda x:getfangan(x["sku_attributes"]),axis=1)
# df["租赁方案"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
# df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"退货拒签"] = np.where(df['物流状态']==5, 1, 0)
df.loc[:,'机型内存'] = df['机型'] + df['内存']

In [20]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    288788
1      9899
Name: count, dtype: int64

# 渠道归属

In [21]:
df.loc[:,"归属渠道"]=df.apply(lambda x:clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"],x['channel_type_id'],x['order_type']),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              167319
搜索渠道                               64983
京东渠道                               33192
单人聊天会话中的小程序消息卡片（分享）                11829
八派信息                                5286
支付宝客户端首页                            3654
支付宝直播                               1696
生活号-J-苹果旗舰                          1448
抖音渠道                                1342
我的小程序入口                             1310
其他渠道场景渠道。                           1235
未知渠道                                1000
派金花                                  911
生活号                                  876
小程序商家消息（服务提醒）                        655
宜品花                                  573
支付宝社群                                480
生活号-商品组件                             156
支群-社群专享                              141
订阅消息                                 132
社群-每日信息首页                            103
曙光计划                                 102
扫描二维码                                 63
顶部搜索框的搜索结果页                           31
九州信息       

# 订单去重

In [22]:
# 保留不去重的数据
df_not_drop = df.copy()
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(
            ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
                , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)
df.shape

(115832, 76)

# 定义状态

In [23]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [24]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()

审核状态
前置拦截       39563
进件前取消      35259
机审强拒       22084
未进件         7270
人审拒绝        6172
出库          2813
已退款         1189
无法联系         715
出库前风控强拒      324
待审核          257
客户取消         178
待归还            4
检测中            2
待商家收货          2
Name: count, dtype: int64

In [25]:
df[df['order_id'] == 3909094]

,create_date,create_time,order_id,order_number,all_money,status,user_id,status2,channel_type,sku_attributes,product_name,new_actual_money,product_id,back_type,user_mobile,true_name,id_card_num,total_describes,decision_result,cancel_reason,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,total_freeze_fund_amount,dy_order_item_json,type,buy_service_product,service_status,order_method,activity_id,appid,max_overdue_days,update_time,reason,purchase_amount,违约金,order_type,union_rent_tag,app_type,已收租金,物流状态,rejected_ato_risk,classify_id,机型,下单日期,月份,date,hour,拒绝理由,取消原因,电审拒绝原因,无法联系原因,商品类型,租赁方案,押金类型,优惠券使用否,是否进行预授权,颜色,内存,进件,来源渠道,退货拒签,机型内存,免审,归属渠道,状态编码,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态
204468,2025-09-11,2025-09-11 14:42:52,3909094,A20250911144243657,8797.0,10,25212522,已退款,其他渠道,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",【预租】iPhone 17 Pro 全新苹果新品 非监管机 顺丰包邮,15396.4,8589,租物归还,18547184580,史儒东,152801199001100018,"[""88888_周汉鸿订单A20250911144243657，msg=拒接"", ""8888...","{""code"":0,""grade"":""S"",""gradeMsg"":""【优质订单】免人审"",""...","88888_周汉鸿审核订单A20250911144243657，审核结果：不通过,msg=其...",b379925efca38fdaa6beb73f5881897f,[],【风控通过】该用户风控资质较好，建议商户可以优先审核发货。为保证订单流程规范，发货前请注意核...,[],1,0,None,None,未知渠道,4,京东活动,14,维客自营账户,NaN,None,7.0,1,NaN,0,414,3F27D10A22175247FBDF164D4C74DA54,NaN,2025-09-11 14:44:07,订单初次分配,NaN,NaN,JD_ORDER,N,JR_APP,NaN,NaN,None,116,iPhone 17 Pro,2025-09-11,9,2025-09-11,14,,NaN,NaN,NaN,全新,租完归还/可买断,全免押,未使用,未预授权,星宇橙色,512GB,进件,未知渠道,0,iPhone 17 Pro512GB,1,京东渠道,3.0,0,0,0,已退款


# 保留商家的数据

In [26]:
df_contain = df.copy()

# 剔除商家数据

In [27]:
df = clean.drop_merchant(df)

# 去重订单明细导出

In [28]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-09-18'

# 导入服务订单

In [29]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(32410, 81)

In [30]:
df = clean.status_node(df)

# 导入台账数据

In [31]:
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025")  
df_ck.shape

(35075, 18)

In [32]:
dfck = pd.merge(df_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)


In [33]:
dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

# dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
try:
    dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)
except:
    dfck.drop(dfck[dfck["类型"]=="露营设备"].index,inplace=True)
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
dfck['日期'] = base_date + pd.to_timedelta(dfck['日期'], unit='D')

# 剔除拒量订单

In [34]:
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2 = df2[df2.是否拒量==0]
day_name_cn_mapping = {
    'Monday': '星期一',
    'Tuesday': '星期二',
    'Wednesday': '星期三',
    'Thursday': '星期四',
    'Friday': '星期五',
    'Saturday': '星期六',
    'Sunday': '星期日'
}

In [35]:
# df_iphone17[df_iphone17['order_id'] == 3909094]

# iPhone 17 全系列

## iPhone17系列总体数据

In [36]:
df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
df_iphone17_all = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, '下单日期')
df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
# 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
df_iphone17_all_new = df_iphone17_all.reset_index()
df_iphone17_all_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-09-10 00:00:00,146.0,41.0,28.08%,37.0,25.34%,16.0,43.24%,21.0,13.0,8.0,21.62%,4.0,2.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,10.81%,35.14%,0.00%,5.41%,0.00%
1,2025-09-11 00:00:00,234.0,69.0,29.49%,81.0,34.62%,55.0,67.90%,26.0,15.0,11.0,13.58%,3.0,5.0,0.0,3.0,0.0,0.0,0.00%,0.00%,0.00%,3.70%,18.52%,0.00%,6.17%,0.00%
2,2025-09-12 00:00:00,218.0,60.0,27.52%,67.0,30.73%,50.0,74.63%,17.0,8.0,9.0,13.43%,3.0,1.0,1.0,4.0,0.0,0.0,0.00%,0.00%,0.00%,4.48%,11.94%,1.49%,1.49%,0.00%
3,2025-09-13 00:00:00,189.0,43.0,22.75%,63.0,33.33%,43.0,68.25%,20.0,14.0,6.0,9.52%,0.0,5.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,22.22%,0.00%,7.94%,0.00%
4,2025-09-14 00:00:00,168.0,41.0,24.40%,47.0,27.98%,34.0,72.34%,13.0,8.0,5.0,10.64%,1.0,0.0,2.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,2.13%,17.02%,4.26%,0.00%,0.00%
5,2025-09-15 00:00:00,231.0,66.0,28.57%,65.0,28.14%,41.0,63.08%,24.0,17.0,7.0,10.77%,1.0,4.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,1.54%,26.15%,0.00%,6.15%,0.00%
6,2025-09-16 00:00:00,254.0,64.0,25.20%,74.0,29.13%,52.0,70.27%,22.0,10.0,12.0,16.22%,3.0,2.0,0.0,7.0,0.0,0.0,0.00%,0.00%,0.00%,4.05%,13.51%,0.00%,2.70%,0.00%
7,2025-09-17 00:00:00,218.0,83.0,38.07%,68.0,31.19%,49.0,72.06%,19.0,12.0,7.0,10.29%,0.0,0.0,0.0,7.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,17.65%,0.00%,0.00%,0.00%
8,2025-09-18 00:00:00,228.0,77.0,33.77%,68.0,29.82%,44.0,64.71%,24.0,14.0,10.0,14.71%,2.0,0.0,2.0,6.0,0.0,0.0,0.00%,0.00%,0.00%,2.94%,20.59%,2.94%,0.00%,0.00%
9,合计,1886.0,544.0,NaN,570.0,NaN,384.0,NaN,186.0,111.0,75.0,NaN,17.0,19.0,5.0,34.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## iPhone17系列租物数据

In [37]:
df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
df_iphone17_zw_group_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-09-10 00:00:00,23.0,12.0,52.17%,5.0,21.74%,1.0,20.00%,4.0,2.0,2.0,40.00%,1.0,1.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,20.00%,40.00%,0.00%,20.00%,0.00%
1,2025-09-11 00:00:00,29.0,12.0,41.38%,4.0,13.79%,3.0,75.00%,1.0,1.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%,0.00%,0.00%
2,2025-09-12 00:00:00,10.0,4.0,40.00%,2.0,20.00%,1.0,50.00%,1.0,1.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,50.00%,0.00%,0.00%,0.00%
3,2025-09-13 00:00:00,12.0,1.0,8.33%,7.0,58.33%,3.0,42.86%,4.0,4.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,57.14%,0.00%,0.00%,0.00%
4,2025-09-14 00:00:00,10.0,5.0,50.00%,1.0,10.00%,1.0,100.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
5,2025-09-15 00:00:00,10.0,6.0,60.00%,1.0,10.00%,0.0,0.00%,1.0,1.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%
6,2025-09-16 00:00:00,20.0,11.0,55.00%,2.0,10.00%,0.0,0.00%,2.0,2.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,100.00%,0.00%,0.00%,0.00%
7,2025-09-17 00:00:00,51.0,28.0,54.90%,9.0,17.65%,5.0,55.56%,4.0,1.0,3.0,33.33%,0.0,0.0,0.0,3.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,11.11%,0.00%,0.00%,0.00%
8,2025-09-18 00:00:00,52.0,26.0,50.00%,8.0,15.38%,5.0,62.50%,3.0,2.0,1.0,12.50%,0.0,0.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%,0.00%,0.00%
9,合计,217.0,105.0,NaN,39.0,NaN,19.0,NaN,20.0,14.0,6.0,NaN,1.0,1.0,0.0,4.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## iPhone17系列搜索数据

In [38]:
df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
df_iphone17_ss_group = all_models.data_group(df_iphone17_ss, df_iphone17_ss_2, df_risk_examine, '下单日期')
df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
df_iphone17_ss_group_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-09-10 00:00:00,43.0,4.0,9.30%,9.0,20.93%,4.0,44.44%,5.0,2.0,3.0,33.33%,0.0,1.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,22.22%,0.00%,11.11%,0.00%
1,2025-09-11 00:00:00,54.0,10.0,18.52%,11.0,20.37%,5.0,45.45%,6.0,5.0,1.0,9.09%,0.0,1.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,45.45%,0.00%,9.09%,0.00%
2,2025-09-12 00:00:00,53.0,7.0,13.21%,13.0,24.53%,7.0,53.85%,6.0,2.0,4.0,30.77%,3.0,0.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,23.08%,15.38%,0.00%,0.00%,0.00%
3,2025-09-13 00:00:00,52.0,4.0,7.69%,15.0,28.85%,10.0,66.67%,5.0,4.0,1.0,6.67%,0.0,0.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,26.67%,0.00%,0.00%,0.00%
4,2025-09-14 00:00:00,46.0,1.0,2.17%,12.0,26.09%,10.0,83.33%,2.0,2.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,16.67%,0.00%,0.00%,0.00%
5,2025-09-15 00:00:00,74.0,11.0,14.86%,15.0,20.27%,10.0,66.67%,5.0,4.0,1.0,6.67%,0.0,1.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,26.67%,0.00%,6.67%,0.00%
6,2025-09-16 00:00:00,92.0,13.0,14.13%,20.0,21.74%,12.0,60.00%,8.0,3.0,5.0,25.00%,2.0,2.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,10.00%,15.00%,0.00%,10.00%,0.00%
7,2025-09-17 00:00:00,63.0,15.0,23.81%,20.0,31.75%,14.0,70.00%,6.0,3.0,3.0,15.00%,0.0,0.0,0.0,3.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,15.00%,0.00%,0.00%,0.00%
8,2025-09-18 00:00:00,61.0,14.0,22.95%,16.0,26.23%,10.0,62.50%,6.0,3.0,3.0,18.75%,1.0,0.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,6.25%,18.75%,0.00%,0.00%,0.00%
9,合计,538.0,79.0,NaN,131.0,NaN,82.0,NaN,49.0,28.0,21.0,NaN,6.0,5.0,0.0,10.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## iPhone17系列京东数据

In [39]:
df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
df_iphone17_jd_group = all_models.data_group(df_iphone17_jd, df_iphone17_jd_2, df_risk_examine, '下单日期')
df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                    "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
df_iphone17_jd_group_new

,下单日期,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,拒量出库,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
0,2025-09-10 00:00:00,15.0,11.0,73.33%,4.0,26.67%,4.0,100.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
1,2025-09-11 00:00:00,88.0,33.0,37.50%,53.0,60.23%,38.0,71.70%,15.0,8.0,7.0,13.21%,3.0,2.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,5.66%,15.09%,0.00%,3.77%,0.00%
2,2025-09-12 00:00:00,75.0,35.0,46.67%,34.0,45.33%,34.0,100.00%,0.0,0.0,0.0,0.00%,0.0,0.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
3,2025-09-13 00:00:00,57.0,24.0,42.11%,29.0,50.88%,23.0,79.31%,6.0,2.0,4.0,13.79%,0.0,4.0,0.0,0.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,6.90%,0.00%,13.79%,0.00%
4,2025-09-14 00:00:00,44.0,25.0,56.82%,17.0,38.64%,13.0,76.47%,4.0,3.0,1.0,5.88%,0.0,0.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,17.65%,0.00%,0.00%,0.00%
5,2025-09-15 00:00:00,61.0,21.0,34.43%,36.0,59.02%,25.0,69.44%,11.0,9.0,2.0,5.56%,0.0,1.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,25.00%,0.00%,2.78%,0.00%
6,2025-09-16 00:00:00,66.0,32.0,48.48%,33.0,50.00%,26.0,78.79%,7.0,3.0,4.0,12.12%,0.0,0.0,0.0,4.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,9.09%,0.00%,0.00%,0.00%
7,2025-09-17 00:00:00,57.0,28.0,49.12%,27.0,47.37%,21.0,77.78%,6.0,5.0,1.0,3.70%,0.0,0.0,0.0,1.0,0.0,0.0,0.00%,0.00%,0.00%,0.00%,18.52%,0.00%,0.00%,0.00%
8,2025-09-18 00:00:00,56.0,23.0,41.07%,33.0,58.93%,24.0,72.73%,9.0,6.0,3.0,9.09%,1.0,0.0,0.0,2.0,0.0,0.0,0.00%,0.00%,0.00%,3.03%,18.18%,0.00%,0.00%,0.00%
9,合计,519.0,232.0,NaN,266.0,NaN,208.0,NaN,58.0,36.0,22.0,NaN,4.0,7.0,0.0,11.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
df_iphone17.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'user_id', 'status2', 'channel_type', 'sku_attributes',
       'product_name', 'new_actual_money', 'product_id', 'back_type',
       'user_mobile', 'true_name', 'id_card_num', 'total_describes',
       'decision_result', 'cancel_reason', 'trace_id', 'rejected', 'result',
       'tips', 'is_vip', 'status_result', 'qvt_risk', 'qvt_result',
       'channel_name', 'channel_type_id', 'activity_name', 'merchant_id',
       'merchant_name', 'total_freeze_fund_amount', 'dy_order_item_json',
       'type', 'buy_service_product', 'service_status', 'order_method',
       'activity_id', 'appid', 'max_overdue_days', 'update_time', 'reason',
       'purchase_amount', '违约金', 'order_type', 'union_rent_tag', 'app_type',
       '已收租金', '物流状态', 'rejected_ato_risk', 'classify_id', '机型', '下单日期', '月份',
       'date', 'hour', '拒绝理由', '取消原因', '电审拒绝原因', '无法联系原因', '商品类型', '租赁方案',
       '押金类型', '优惠券使用否', '是否进行预授权', '颜

In [41]:
# df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17_2['已退款']==1)]

## iPhone17系列客户取消数据

In [42]:
df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
df_iphone17_qx

,order_number,status2,归属渠道,是否拒量,cancel_reason,取消原因,拒绝理由
187503,A20250910142926502,已退款,芝麻租物,0,"88888_张丹琦审核订单A20250910142926502，审核结果：不通过,msg=其...",NaN,
188875,A20250910152227416,已退款,支付宝客户端首页,0,"88888_廖丽敏审核订单A20250910152227416，审核结果：不通过,msg=其...",NaN,
189431,A20250910154259784,已退款,支付宝社群,0,"88888_廖丽敏审核订单A20250910154259784，审核结果：不通过,msg=其...",NaN,
196414,A20250910234325125,已退款,其他渠道场景渠道。,0,"88888_廖丽敏审核订单A20250910234325125，审核结果：不通过,msg=其...",NaN,
203313,A20250911135632595,已退款,京东渠道,0,"88888_胡彩滢审核订单A20250911135632595，审核结果：不通过,msg=其...",NaN,
204468,A20250911144243657,已退款,京东渠道,0,"88888_周汉鸿审核订单A20250911144243657，审核结果：不通过,msg=其...",NaN,
212579,A20250911224328167,已退款,京东渠道,0,"88888_廖丽敏审核订单A20250911224328167，审核结果：不通过,msg=其...",NaN,
213993,A2025091208170531,已退款,搜索渠道,0,"88888_黄兰娟审核订单A2025091208170531，审核结果：不通过,msg=其他...",NaN,
222712,A2025091217040358,已退款,搜索渠道,0,"88888_邹巧巧审核订单A2025091217040358，审核结果：不通过,msg=其他...",NaN,
225995,A20250912205554326,已退款,搜索渠道,0,"88888_刘三妹审核订单A20250912205554326，审核结果：不通过,msg=其...",NaN,


## iPhone17系列无法联系数据

In [43]:
df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
df_iphone17_wf


,order_number,status2,归属渠道,是否拒量,cancel_reason,取消原因,拒绝理由
188586,A20250910151306232,已退款,搜索渠道,0,"88888_廖丽敏审核订单A20250910151306232，审核结果：不通过,msg=用...",NaN,
195289,A20250910204704286,已退款,芝麻租物,0,"88888_何静审核订单A20250910204704286，审核结果：不通过,msg=用户...",NaN,
196710,A20250911005639142,已退款,支付宝社群,0,"88888_冯二洋审核订单A20250911005639142，审核结果：不通过,msg=用...",NaN,
197181,A2025091103134219,已退款,未知渠道,0,"88888_冯二洋审核订单A2025091103134219，审核结果：不通过,msg=用户...",NaN,
198157,A2025091109254276,已退款,京东渠道,0,"88888_邹巧巧审核订单A2025091109254276，审核结果：不通过,msg=用户...",NaN,
198999,A20250911102403279,已退款,京东渠道,0,"88888_冯二洋审核订单A20250911102403279，审核结果：不通过,msg=用...",NaN,
205769,A20250911153159562,已退款,搜索渠道,0,"88888_冯二洋审核订单A20250911153159562，审核结果：不通过,msg=用...",NaN,
221803,A20250912162353321,已退款,未知渠道,0,"88888_邹巧巧审核订单A20250912162353321，审核结果：不通过,msg=用...",NaN,
227852,A2025091305181618,已退款,京东渠道,0,"88888_邹巧巧审核订单A2025091305181618，审核结果：不通过,msg=用户...",NaN,
230457,A20250913115705585,已退款,京东渠道,0,"88888_何静审核订单A20250913115705585，审核结果：不通过,msg=用户...",NaN,


# iPhone 16 的遗老遗少可能有用

In [44]:

# df_16 = df[df['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_16 = df_16[df_16['product_name']!='iPhone 16 2024年新机 200元权益包']
# df_16['product_name'] = df_16['product_name'].str.replace(' ', '', regex=False)
# # df_16['型号内存'] = df_16['product_name'].str.split(' 国').str.get(0).str.split(' 2').str.get(0).str.replace(' ', '', regex=False).str.replace('首月5折', '', regex=False)
# df_16['型号内存'] = df_16['product_name'].str.extract(r'(iPhone ?16(ProMax|Pro|Plus)?)', expand=True)[0]

# def iPhone(df, qd_names):
#     # df_16_group = df_16.groupby(['归属渠道','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
#     df_16_group = df.groupby([f'{qd_names}','型号内存', '下单日期']).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum','机审强拒':'sum','人审拒绝':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})

#     df_16_group.rename(columns={'order_id':'去重订单数'},inplace=True )

#     df_16_group["拦截率"] = df_16_group["前置拦截"]/df_16_group["去重订单数"]
#     df_16_group["拦截率"] = df_16_group["拦截率"].apply(lambda x:format(x,".2%"))

#     df_16_group['预授权通过率'] = df_16_group["是否进件"]/df_16_group["去重订单数"]
#     df_16_group["预授权通过率"] = df_16_group["预授权通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["强拒比例"] = df_16_group["机审强拒"]/df_16_group["是否进件"]
#     df_16_group["强拒比例"] = df_16_group["强拒比例"].apply(lambda x:format(x,".2%"))

#     df_16_group["机审通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]

#     df_16_group["风控通过件"] = df_16_group["是否进件"]-df_16_group["机审强拒"]-df_16_group["人审拒绝"]
#     df_16_group["风控通过率"] = df_16_group["风控通过件"]/df_16_group["是否进件"]
#     df_16_group["风控通过率"] = df_16_group["风控通过率"].apply(lambda x:format(x,".2%"))

#     df_16_group["取消率"] = df_16_group["客户取消"]/df_16_group["是否进件"]
#     df_16_group["取消率"] = df_16_group["取消率"].apply(lambda x:format(x,".2%"))


#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝"]/df_16_group["是否进件"]
#     df_16_group["人审拒绝率"]=df_16_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

#     df_16_group["进件出库率"]=df_16_group["是否出库"]/df_16_group["是否进件"]
#     df_16_group["进件出库率"]=df_16_group["进件出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group["订单出库率"]=df_16_group["是否出库"]/df_16_group["去重订单数"]
#     df_16_group["订单出库率"]=df_16_group["订单出库率"].apply(lambda x:format(x,".2%"))

#     df_16_group = df_16_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系","出库前风控强拒","待审核"]]
#     df_16_group
#     return df_16_group


In [45]:
# # 计算环比
# def calculate_growth_rate(current, previous):
#     if previous == 0:
#         return float('inf') if current > 0 else float('-inf')
#     return (current - previous) / previous

In [46]:
# # 获取渠道的具体渠道
# qd_names = ['来源渠道', '归属渠道']
# indx1 = 1
# qd_group = iPhone(df_16, qd_names[indx1])
# qd_lists = ['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']
# qd_list =  qd_group.reset_index()[qd_names[indx1]][qd_group.reset_index()[qd_names[indx1]].isin(qd_lists)].drop_duplicates().to_list()


In [47]:
# # 渠道明细 去重订单数
# qd_type = ['去重订单数', '是否进件']
# indx2 = 0
# df_16_group_qcdd = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_qcdd = df_16_group_qcdd.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_qcdd = df_16_group_qcdd.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_qcdd = df_pivot_qcdd[ qd_list].reset_index()

In [48]:
# # 渠道明细 是否进件
# indx2 = 1
# df_16_group_jj = qd_group.groupby([qd_names[indx1], '下单日期']).agg({qd_type[indx2]: 'sum'}).reset_index()
# df_16_group_jj = df_16_group_jj.sort_values(by=[qd_names[indx1], '下单日期'], ascending=(False, False))
# df_pivot_jj = df_16_group_jj.pivot(index='下单日期', columns=qd_names[indx1], values=qd_type[indx2])
# df_pivot_jj = df_pivot_jj[ qd_list].reset_index()

In [49]:
# # 渠道数据汇总
# # 提取最后两行的下单日期用于对比
# current = df_pivot_jj['下单日期'].iloc[-1].strftime('%m%d')
# previous = df_pivot_jj['下单日期'].iloc[-2].strftime('%m%d')
# df_16_qcdd = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# df_16_jj = pd.DataFrame({'下单日期': [f'{current}与{previous}对比增长数量', f'{current}与{previous}对比增长率']})
# for qd_name in qd_list:
#     df_16_qcdd[qd_name] =  [df_pivot_qcdd[qd_name].diff().iloc[-1],  (df_pivot_qcdd[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]
#     df_16_jj[qd_name] =  [df_pivot_jj[qd_name].diff().iloc[-1],  (df_pivot_jj[qd_name].pct_change() * 100).apply(lambda x: f'{x:.0f}%').iloc[-1]]

# indx = 0
# df_pivot_qcdds = pd.concat([df_pivot_qcdd[indx:], df_16_qcdd])
# df_pivot_jjs = pd.concat([df_pivot_jj[indx:], df_16_jj])
# # df_pivot_jjs

# 设置定时任务

In [50]:
from apscheduler.schedulers.blocking import BlockingScheduler
from apscheduler.schedulers.background import BackgroundScheduler
import gc
hour = 17
minute = 30
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
def my_job1():
    
    # print(f'执行定时任务：现在是每日的{H}点{M}分')
    path1 = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
    Today1 = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today1}的{hour}点{minute}分')
    
    df_order = query(sql1)
    df_risk_examine = query(sql3)
    # 数据处理
    df, df2 = process_data(df_order)
    
    #iphone 17系列转化数据，总体
    df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
    df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
    df_iphone17_all = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, '下单日期')
    df_iphone17_all = df_iphone17_all[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    # 设置汇总行对数值列进行汇总，字符列或类型为object的列不做处理输出为nan
    df_iphone17_all.loc['合计'] = df_iphone17_all.select_dtypes(include=[np.number]).sum()
    df_iphone17_all_new = df_iphone17_all.reset_index()
    
    # 租物
    df_iphone17_zw = df_iphone17[df_iphone17['归属渠道']=='芝麻租物']
    df_iphone17_zw_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='芝麻租物']
    df_iphone17_zw_group = all_models.data_group(df_iphone17_zw, df_iphone17_zw_2, df_risk_examine, '下单日期')
    df_iphone17_zw_group = df_iphone17_zw_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_zw_group.loc['合计'] = df_iphone17_zw_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_zw_group_new = df_iphone17_zw_group.reset_index()
    
    # 搜索渠道
    df_iphone17_ss = df_iphone17[df_iphone17['归属渠道']=='搜索渠道']
    df_iphone17_ss_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='搜索渠道']
    df_iphone17_ss_group = all_models.data_group(df_iphone17_ss, df_iphone17_ss_2, df_risk_examine, '下单日期')
    df_iphone17_ss_group = df_iphone17_ss_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_ss_group.loc['合计'] = df_iphone17_ss_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_ss_group_new = df_iphone17_ss_group.reset_index()
    
    # 京东渠道
    df_iphone17_jd = df_iphone17[df_iphone17['归属渠道']=='京东渠道']
    df_iphone17_jd_2 = df_iphone17_2[df_iphone17_2['归属渠道']=='京东渠道']
    df_iphone17_jd_group = all_models.data_group(df_iphone17_jd, df_iphone17_jd_2, df_risk_examine, '下单日期')
    df_iphone17_jd_group = df_iphone17_jd_group[["去重订单数","前置拦截","拦截率","进件数","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝","风控通过件","风控通过率","客户取消","无法联系",
                        "出库前风控强拒","待审核",'出库','拒量出库', '进件出库率','总体进件出库率（含拒量）','拒量进件出库率增加','取消率','人审拒绝率','出库前强拒比例','无法联系占比','订单出库率']].fillna(0)
    df_iphone17_jd_group.loc['合计'] = df_iphone17_jd_group.select_dtypes(include=[np.number]).sum()
    df_iphone17_jd_group_new = df_iphone17_jd_group.reset_index()
    
    # 客户取消数据
    df_iphone17_qx = df_iphone17[(df_iphone17['客户取消']==1)|(df_iphone17['已退款']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]

    # 无法联系数据
    df_iphone17_wf = df_iphone17[(df_iphone17['无法联系']==1)][['order_number', 'status2','归属渠道', '是否拒量','cancel_reason', '取消原因', '拒绝理由']]
    
    
    with pd.ExcelWriter(path1 + f'iPhone 17系列_{Today1}.xlsx', engine='openpyxl') as writer:
        df_iphone17_all_new.to_excel(writer, sheet_name='iPhone17系列总体', index=False)
        df_iphone17_zw_group_new.to_excel(writer, sheet_name='iPhone17系列芝麻租物', index=False)
        df_iphone17_ss_group_new.to_excel(writer, sheet_name='iPhone17系列搜索渠道', index=False)
        df_iphone17_jd_group_new.to_excel(writer, sheet_name='iPhone17系列京东渠道', index=False)
        df_iphone17_qx.to_excel(writer, sheet_name='iPhone17系列取消明细', index=False)
        df_iphone17_wf.to_excel(writer, sheet_name='iPhone17系列无法联系明细', index=False)
    del df_iphone17_all_new,df_iphone17_zw_group_new,df_iphone17_ss_group_new,df_iphone17_jd_group_new,df_iphone17_qx,df_iphone17_wf
    gc.collect()
    print("iPhone 17系列数据已保存,内存已回收")
    # with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
    #     df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
    #     qd_group.to_excel(writer, sheet_name='iPhone16系列明细')

scheduler1 = BackgroundScheduler()
# 添加一个cron任务，每天的17点30分执行
scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)
my_job1()
# 启动scheduler1
scheduler1.start()


执行定时任务：现在是2025091817的17点30分


iPhone 17系列数据已保存,内存已回收


In [51]:
# df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
# df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
# # 筛选支付宝和京东的数据
# df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
# df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
# df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
# df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
# df_iphone17_group = df_iphone17_group[['进件数', '出库']]
# # 计算进件占比与出库占比
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
# df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
# df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
# df_iphone17_group

In [52]:
# # iPhone16系列、iPhone17系列占比情况更新
# # 机型+内存+渠道（支付宝、京东）+进件+出库+进件占比+出库占比，
# # 每日数据更新 至17预租数据文件内。
# def myjob2():
#     minute = 35
#     path = r'\\digua\迪瓜租机\18.iPhone17预租数据/'
#     Today = str(datetime.now().strftime('%Y%m%d%H'))
#     print(f'执行定时任务：现在是{Today}的{minute}分')
    
#     df_order = query(sql1)
#     df_risk_examine = query(sql3)
#     # 数据处理
#     df, df2 = process_data(df_order)
#     df.loc[:, '订单渠道'] = np.where(df['order_type']=='ZFB_ORDER', '支付宝', df['归属渠道'])
#     df2.loc[:, '订单渠道'] = np.where(df2['order_type']=='ZFB_ORDER', '支付宝', df2['归属渠道'])
#     # 筛选支付宝和京东的数据
#     df = df[(df['order_type']=='ZFB_ORDER')|(df['归属渠道']=='京东渠道')]
#     df2 = df2[(df2['order_type']=='ZFB_ORDER')|(df2['归属渠道']=='京东渠道')]
    
#     # 筛选iphone16系列数据
#     df_iphone16 = df[df.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_2 = df2[df2.product_name.str.contains(r'iPhone 16|iPhone16')]
#     df_iphone16_group = all_models.data_group(df_iphone16, df_iphone16_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone16_group = df_iphone16_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件数'] / df_iphone16_group['进件数'].sum()
#     df_iphone16_group.loc[:, '进件占比'] = df_iphone16_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库'] / df_iphone16_group['出库'].sum()
#     df_iphone16_group.loc[:, '出库占比'] = df_iphone16_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     #iphone 17系列转化数据，总体
#     df_iphone17 = df[df['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_2 = df2[df2['product_name'].str.contains(r'iPhone 17|iPhone Air')]
#     df_iphone17_group = all_models.data_group(df_iphone17, df_iphone17_2, df_risk_examine, ['机型内存', '订单渠道'])
#     df_iphone17_group = df_iphone17_group[['进件数', '出库']]
#     # 计算进件占比与出库占比
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件数'] / df_iphone17_group['进件数'].sum()
#     df_iphone17_group.loc[:, '进件占比'] = df_iphone17_group['进件占比'].apply(lambda x:format(x,".2%"))
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库'] / df_iphone17_group['出库'].sum()
#     df_iphone17_group.loc[:, '出库占比'] = df_iphone17_group['出库占比'].apply(lambda x:format(x,".2%"))
    
#     with pd.ExcelWriter(path + f'iPhone 16-17系列占比_{Today}.xlsx', engine='openpyxl') as writer:
#         df_iphone16_group.to_excel(writer, sheet_name='iPhone16系列占比', index=True)
#         df_iphone17_group.to_excel(writer, sheet_name='iPhone17系列占比', index=True)
        
#     del df_iphone16_group,df_iphone17_group
#     gc.collect()
#     print("iPhone 16、17系列占比数据已保存,内存已回收")


# myjob2()
# scheduler2 = BackgroundScheduler()
# # 添加一个cron任务，每天的17点35分执行
# scheduler2.add_job(myjob2, 'cron', hour=17, minute=35)

# # 启动scheduler1
# scheduler2.start()


In [53]:
# ip16 = pd.read_excel(r'\\digua\迪瓜租机\18.iPhone16预租数据/iPhone 16系列_2024092018.xlsx')
# ip16 = ip16.ffill()
# ip16['下单日期2'] = pd.to_datetime(ip16['下单日期']).dt.date
# ip16_group = ip16.groupby(['来源渠道', '下单日期2']).agg({'去重订单数': 'sum', '是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['来源渠道', '下单日期2'], ascending=(False, False))
# # ip16_group = ip16_group.sort_values(by=['下单日期2'], ascending=(False))
# # with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
# #         ip16_group.to_excel(writer, sheet_name='iPhone16系列转化')
# ip16_group

In [54]:

# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'去重订单数': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='去重订单数')
# df_pivot1 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl') as writer:
#         df_pivot1.to_excel(writer, sheet_name='iPhone16系列转化(去重订单数)')
# # ip16_group

In [55]:
# ip16['去重订单数'].sum()

In [56]:
# 
# ip16_group['去重订单数'].sum()

In [57]:
# ip16 = df_16_group
# ip16_group = ip16.groupby(['归属渠道', '下单日期']).agg({'是否进件': 'sum'}).reset_index()
# ip16_group = ip16_group.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot = ip16_group.pivot(index='下单日期', columns='归属渠道', values='是否进件')
# df_pivot2 = df_pivot[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# with pd.ExcelWriter('F:/iPhone 16系列.xlsx', engine='openpyxl', mode='a') as writer:
#         df_pivot2.to_excel(writer, sheet_name='iPhone16系列转化(进件)')

# 待审核

In [58]:
# df_dsh = df[df['待审核']==1]
# df_dsh = df_dsh[df_dsh['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_dsh = df_dsh[['order_number', 'create_time', 'product_name', '租赁方案', '内存', '颜色', '来源渠道']]
# df_dsh.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'product_name': '型号'}, inplace=True)
# df_dsh


# with pd.ExcelWriter('F:/需求/iPhone16.xlsx', engine='openpyxl') as writer:
#     df_dsh.to_excel(writer, sheet_name='待审核', index=False)